## IMPORTER LES BIBLIOTHEQUES

In [1]:
from py2neo import Graph
import pymongo

## FONCTION DE CONECTION A MONGODB

In [2]:
def connectToMongoDB():
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["socialnet"]
    return mydb

## FONCTION DE CONNECTION A NEO4J

In [3]:
def connectToNeo4j(user, password):
    '''
    Etablie une connection avec Neo4j.
    le username par défaut est is 'neo4j'
    '''
    url = "bolt://localhost:7687"
    try:
        connection = Graph(url, user=user, password=password)
    except:
        connection = Graph(url, user=user, password=password)
    return connection

## FONCTION POUR EVITER DES DOUBLONS

In [13]:
def removeExistingData(connection):
    '''
    Neo4j does not replace a dataset when additional data is added, that is
    why we need to remove all existing data before loading the dataset again
    '''
    connection.run("MATCH (n) DETACH DELETE n")

## FONCTION QUI REPREND LES FONCTIONS PRECEDENTES ET ETABLIT LES LIENS ET LES NOEUDS SUR NEO4J

In [6]:
def main():
    graph = connectToNeo4j("neo4j", "Peg@sus01")
    removeExistingData(graph)
    my_mongodb = connectToMongoDB()

    col_users = my_mongodb["users"] #Collection des Utilisateurs
    col_posts = my_mongodb["posts"] #Collection Publications

    list_users = col_users.find()
    list_posts = col_posts.find()

    for user in list_users:
        #Crétion du noeud utilisateur
        user_alias ="User_"+user['_id']
        query ="CREATE ("+user_alias+":Person {name: $name, firstname : $fname, lastname: $lname, birthday: $bday, identifier: $id})"
        graph.run(query,parameters= { "name" : user['pseudo'],"fname":user['firstname'], "lname":user['lastname'], "bday" : user['birthday'], "id":user['_id']})
        
        #creation des liens d'amitíé
        for friend_id in user['friends']:
            query = "RETURN exists( (:Person {identifier: $id_userA})-[:AMI]-(:Person {identifier: $id_userB}) )"
           
            #Verification de relation dans les deux sens 
            condition_1 = graph.run(query,parameters = {"id_userA": user['_id'] , "id_userB": friend_id}).evaluate()
            condition_2 = graph.run(query,parameters = {"id_userA": friend_id , "id_userB": user['_id']}).evaluate()
            
            #Creation du lien d'amitié si aucun lien existe
            if not(condition_1 or condition_2): #Vrai exclusivement si les deux conditions sont fausses
                query1 = "MATCH (a:Person),(b:Person) WHERE a.identifier = $id_user AND b.identifier = $id_friend CREATE (a)-[r:AMI]->(b) RETURN r"
                graph.run(query1, parameters = {"id_user":user['_id'],"id_friend":friend_id})


    for post in list_posts:
        #Création du noeud de la publication
        title = "Post_"+post['_id']
        query1 ="CREATE ("+title+":Post {title:$title, images : $img, content: $content, likes: $nbre_like, date: $date})"       
        graph.run(query1,parameters= { "title":title, "img": post['images'],"content": post['content'], "nbre_like": len(post['likes']), "date": post['date']})
        
        #Création de la relation publier
        query2 = "MATCH (a:Person),(b:Post) WHERE a.identifier = $id AND b.title = $title CREATE (a)-[r:PUBLIER]->(b) RETURN r"
        graph.run(query2, parameters = {"id":post['id_author'],"title":title})

        #Création de la relation publier
        for user_id in post['likes']:
            query = "MATCH (a:Person),(b:Post) WHERE a.identifier = $id AND b.title = $title CREATE (a)-[r:LIKER]->(b) RETURN r"
            graph.run(query, parameters = {"id":user_id,"title":title})
        
        #Création de la relation commenter
        for comment in post['comment']:
            query = "MATCH (a:Person),(b:Post) WHERE a.identifier = $id AND b.title = $title CREATE (a)-[r:COMMENTER  {content: $content, date: $date}]->(b) RETURN r"
            graph.run(query, parameters = {"id":comment['id_user'],"title":title, "content": comment['comment'], "date": comment['date']})


In [10]:
main()

In [9]:
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
DELETE n,r

SyntaxError: invalid syntax (<ipython-input-9-5963be4b5130>, line 2)